In [1]:
# Required to access the database
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Data analysis tools
import pandas as pd
import numpy as np
import seaborn as sns

# Models available in our application
from datasets.models import RawFlower, RawUNM, RawDAR
from django.contrib.auth.models import User
from datasets.models import RawDictionary


from datasets.models import RawNEU
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels

!pip install lxml

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
from api import adapters
from api import analysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from api import dilutionproc   

In [4]:
def merge2CohortFrames2(df1,df2):
    'merge on feature intersections'

    for as_feature in ['UASB', 'UDMA', 'UAS5', 'UIAS', 'UAS3', 'UMMA']:
        if as_feature not in df1.columns:
            df1[as_feature] = np.nan
        if as_feature not in df2.columns:
            df2[as_feature] = np.nan

    s1 = set(df1.columns)
    s2 = set(df2.columns)
 

    cc = set.intersection(s1, s2)

    df_all = pd.concat([df1[cc],df2[cc]])

    return df_all


def merge3CohortFrames2(df1,df2,df3):
    'merge on feature intersections'

    for as_feature in ['UASB', 'UDMA', 'UAS5', 'UIAS', 'UAS3', 'UMMA']:
        if as_feature not in df1.columns:
            df1[as_feature] = np.nan
        if as_feature not in df2.columns:
            df2[as_feature] = np.nan
        if as_feature not in df3.columns:
            df3[as_feature] = np.nan

    s1 = set(df1.columns)
    s2 = set(df2.columns)
    s3 = set(df3.columns)

    cc = set.intersection(s1, s2, s3)

    df_all = pd.concat([df1[cc],df2[cc],df3[cc]])

    return df_all

In [5]:
## decide whether to exclude fish consumption (True = include fish, False = No fish)
#Model1 = False
#Model4 = True
fish_consumption = True
## decide whether to adjust the model using the dilution procedure (True = run dilution adj, False = no)
run_adjustment = False
## specify whether to use dilution as covariate (only works if two cohorts/summation is run)
use_dilution_as_covar = True


if use_dilution_as_covar == True:
    run_adjustment = False

## exposure var

#exposure_var = 'SUM2'
#exposure_var = 'UIAS'
exposure_var = 'UIAS'
exposure_var = 'UIASMMADMA'

if 'UIAS' in exposure_var:
    fish_consumption = True

In [6]:
## Get the data

## Get NEU data with no fish
df_NEU = adapters.neu.get_dataframe_orig()
df_NEU = df_NEU[df_NEU['TimePeriod']==2] # Visit 2

df_NEU_covars = adapters.neu.get_dataframe_covars()
df_NEU = df_NEU_covars.merge(df_NEU, on = ['PIN_Patient','CohortType','TimePeriod']) #Merge the covariates
df_NEU = df_NEU.replace(-9,np.nan).replace('-9', np.nan)
## Get DAR data
df_DAR = adapters.dar.get_dataframe()

#df_DAR = df_NEU.copy()
#df_DAR['CohortType'] = 'DAR'

## Get UNM data with no fis

df_UNM = adapters.unm.get_dataframe_orig()
df_UNM_covars = adapters.unm.get_dataframe_covars()
df_UNM = df_UNM_covars.merge(df_UNM, on = ['PIN_Patient','CohortType','TimePeriod']) #Merge the covariates



## Adjustments
##*** TAKE OUT ****#
  
mu1, sigma1 = 0, .2
mu2, sigma2 = 0, .15

df_UNM['UAS5'] = np.random.normal(mu1, sigma1, df_UNM.shape[0])
df_UNM['UAS3'] = np.random.normal(mu1, sigma1, df_UNM.shape[0])
df_UNM['UMMA'] = np.random.normal(mu1, sigma1, df_UNM.shape[0])
df_UNM['UDMA'] = np.random.normal(mu1, sigma1, df_UNM.shape[0])
#df_DAR['UAS5'] = np.random.normal(mu1, sigma1, df_DAR.shape[0])
#df_DAR['UAS3'] = np.random.normal(mu1, sigma1, df_DAR.shape[0])


for conc_var in ['UTAS','UIAS','UMMA','UDMA','UAS3','UAS5']:
    df_DAR[conc_var] = df_DAR[conc_var] * (np.nanmedian(df_DAR['urine_specific_gravity']) - 1) / (df_DAR['urine_specific_gravity']-1)

for conc_var in ['UTAS','UMMA','UDMA','UAS3','UAS5']:
    df_UNM[conc_var] = df_UNM[conc_var] * (np.nanmedian(df_UNM['creatininemgdl']) - 1) / (df_UNM['creatininemgdl']-1)
    #df_UNM[conc_var] = df_UNM[conc_var] * (np.nanmedian(df_UNM['urine_specific_gravity']) - 1) / (df_UNM['urine_specific_gravity']-1)

for conc_var in ['UTAS']:
    df_NEU[conc_var] = df_NEU[conc_var] * (np.nanmedian(df_NEU['SPECIFICGRAVITY_V2']) - 1) / (df_NEU['SPECIFICGRAVITY_V2']-1)

## comment these two lines if on live machine
#df_UNM = df_DAR.copy()
#df_UNM['CohortType'] = 'UNM'

#df_UNM = df_NEU.copy()


# Remove fish
if fish_consumption == False:
    df_NEU = df_NEU[(df_NEU['fish_pu_v2'] == 0) & (df_NEU['fish'] == 0)] #No fish consumption
    df_UNM = df_UNM[df_UNM['fish']==0]
    df_DAR = adapters.dar.get_dataframe_nofish()

 # Total sum all 3 cohorts are being run since they all have 'harmonized' UTAS   
if exposure_var == 'UTAS':
    
    df_ALL = merge3CohortFrames2(df_NEU, df_UNM, df_DAR)

    frames_for_adjust = [
        ('NEU', df_NEU),
        ('UNM', df_UNM),
        ('DAR', df_DAR)
    ]

    #df_ALL = analysis.merge3CohortFrames(df_NEU, df_UNM, df_DAR)
    frames_for_analysis = [
        ('NEU', df_NEU),
        ('UNM', df_UNM),
        ('DAR', df_DAR),
        ('ALL', df_ALL)

    ]

    for name, df in frames_for_analysis:
        print('Data Stats')
        print(name)
        print(df.shape)

# this is for the speciatated summations. NEU does not have this data, only DAR and UNM
# two different sums are being considered
if exposure_var == 'UIAS' or exposure_var == 'UIASMMADMA':
    
    #df_ALL is now only 2 cohorts

    frames_for_adjust = [
        ('UNM', df_UNM),
        ('DAR', df_DAR)
    ]
    
    #TODO: Calculate the summation?
    #['UASB', 'UDMA', 'UAS5', 'UIAS', 'UAS3', 'UMMA']

    
    #sum the speciations
    #SUM1 = UAS5 + UAS3
    #SUM2 = UAS5 + UAS3 + UMMA + UDMA
    if exposure_var == 'UIAS':
        
        sum_cond_dar = ((~df_DAR['UAS5'].isna()) & (~df_DAR['UAS3'].isna())) 
        sum_cond_unm = ((~df_UNM['UAS5'].isna()) & (~df_UNM['UAS3'].isna())) 
        
        #df_DAR['SUM1'] = np.where(sum_cond_dar, np.sum(df_DAR[['UAS5','UAS3']], axis=1), np.nan)
        df_DAR['UIAS'] = df_DAR['UIAS']
        df_UNM['UIAS'] = np.where(sum_cond_unm, np.sum(df_UNM[['UAS5','UAS3']], axis=1), np.nan)
        
    if exposure_var == 'UIASMMADMA':
        #make sure nans dont get summed
        #sum_cond_dar = ((~df_DAR['UAS5'].isna()) & (~df_DAR['UAS3'].isna()) & (~df_DAR['UMMA'].isna()) & (~df_DAR['UDMA'].isna())) 
        sum_cond_dar = ((~df_DAR['UIAS'].isna()) & (~df_DAR['UMMA'].isna()) & (~df_DAR['UDMA'].isna())) 

        sum_cond_unm = ((~df_UNM['UAS5'].isna()) & (~df_UNM['UAS3'].isna()) & (~df_UNM['UMMA'].isna()) & (~df_UNM['UDMA'].isna())) 
        
        #df_DAR['SUM2'] = np.where(sum_cond_dar, np.sum(df_DAR[['UAS5','UAS3','UMMA','UDMA']], axis=1), np.nan)
        df_DAR['UIASMMADMA'] = np.where(sum_cond_dar, np.sum(df_DAR[['UIAS','UMMA','UDMA']], axis=1), np.nan)
        df_UNM['UIASMMADMA'] = np.where(sum_cond_unm, np.sum(df_UNM[['UAS5','UAS3','UMMA','UDMA']], axis=1), np.nan)
        
    
    #merge the cohorts
    df_ALL = merge2CohortFrames2(df_UNM, df_DAR)

    frames_for_analysis = [
        ('UNM', df_UNM),
        ('DAR', df_DAR),
        ('ALL', df_ALL)

    ]
    
    for name, df in frames_for_analysis:
        print('Data Stats')
        print(name)
        print(df.shape)


Data Stats
UNM
(135, 36)
Data Stats
DAR
(2152, 199)
Data Stats
ALL
(2287, 34)


df_DAR2 = pd.DataFrame.from_records(
    RawDAR.objects.
    # exclude(Creat_Corr_Result__lt=-1000).
    # exclude(Creat_Corr_Result__isnull=True).
    values()
)

In [7]:
#for x in enumerate(df_DAR2.columns):
#    print(x)

In [8]:
#for x in enumerate(df_DAR.columns):
#    if(x[1] == 'UTAS'):
#        print(x)

In [9]:
#adapters.dar.get_dataframe()['UTAS'].describe()

In [10]:
#df_DAR['UTAS'].describe()

In [11]:
#df_DAR2['As'].describe()

In [12]:
df_DAR['birthWt'].describe()

count    1930.000000
mean     3413.505365
std       540.390874
min       357.203700
25%      3125.000000
50%      3430.275500
75%      3741.500000
max      5400.000000
Name: birthWt, dtype: float64

In [13]:
sum_cond_dar = ((~df_DAR['UAS5'].isna()) & (~df_DAR['UAS3'].isna())) 

df_DAR['SUM1'] = np.where(sum_cond_dar, np.sum(df_DAR[['UAS5','UAS3']], axis=1), np.nan)

df_DAR[['SUM1','UIAS']]

,SUM1,UIAS
0,0.177418,0.177418
1,4.742500,4.742500
2,NaN,NaN
3,0.042073,0.042073
4,0.306808,0.306808
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,0.530387,0.530387


In [14]:
#df_DAR2['urine'].describe()

for x in df_DAR:
    print(x)

id
PIN_Patient
assay
Member_c
TimePeriod
sample_gestage_days
Outcome
age
ethnicity
race
education
BMI
smoking
parity
preg_complications
folic_acid_supp
babySex
birth_year
birth_month
birthWt
birthLen
headCirc
ponderal
PNFFQTUNA
PNFFQFR_FISH_KIDS
PNFFQSHRIMP_CKD
PNFFQDK_FISH
PNFFQOTH_FISH
mfsp_6
TOTALFISH_SERV
fish
folic_acid
income5y
urine_batchno_bulk
urine_batchno_spec
collect_age_days
collect_age_src
collection_season
pH
UTASProp
PropMMAtoiAs
PropDMAtoMMA
urine_specific_gravity
WeightZScore
WeightCentile
LGA
SGA
GDMTest1Ag
GDMTest2Age
GDMtest1
GDMtest2
UIAS
iAs_IDL
iAs_BDL
iAs_N
UASB
AsB_IDL
AsB_BDL
AsB__N
UAS3
AsIII_IDL
AsIII_BDL
AsIII_N
UAS5
AsV_IDL
AsV_BDL
AsV_N
UDMA
DMA_IDL
DMA_BDL
DMA_N
UMMA
MMA_IDL
MMA_BDL
MMA_N
UBA
Ba_IDL
Ba_BDL
Ba_N
UAG
Ag_IDL
Ag_BDL
Ag_N
UAL
Al_IDL
Al_BDL
Al_N
UTAS
As_IDL
As_BDL
As_N
UBE
Be_IDL
Be_BDL
Be_N
UCA
Ca_IDL
Ca_BDL
Ca_N
UCD
Cd_IDL
Cd_BDL
Cd_N
UCO
Co_IDL
Co_BDL
Co_N
UCR
Cr_IDL
Cr_BDL
Cr_N
UCS
Cs_IDL
Cs_BDL
Cs_N
UCU
Cu_IDL
Cu_BDL
Cu_N
UFE
Fe_IDL
Fe_B

dsfsfdsfd

dat <- read.csv('dfUNM_wide.csv') %>%
  mutate(
    UTAS_cradj = UTAS * (median(creatininemgdl, na.rm = TRUE)-1)/(creatininemgdl-1)
  , UMMA_cradj = UMMA * (median(creatininemgdl, na.rm = TRUE)-1)/(creatininemgdl-1)
  , UDMA_cradj = UDMA * (median(creatininemgdl, na.rm = TRUE)-1)/(creatininemgdl-1)
  , UAS3_cradj = UAS3 * (median(creatininemgdl, na.rm = TRUE)-1)/(creatininemgdl-1)
  , UAS5_cradj = UAS5 * (median(creatininemgdl, na.rm = TRUE)-1)/(creatininemgdl-1)
  )

In [15]:
#df_DAR.to_csv('DAR_data.csv', index = False)
#df_NEU.to_csv('NEU_data.csv', index = False)

In [16]:
import os
import shutil


#directories to store the results based on combination of fish and adjustment procedurs. If exists, delete.
#models below will use this to output their results
directories_for_output = ['results/' + x for x in ['rresultslme','rresultslmer','rresultsglm','rresultsglmer']]

for directory in directories_for_output:

    try:
        shutil.rmtree(directory + '_' + str(run_adjustment) + '_' + str(fish_consumption) + '_' + str(exposure_var))
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))
    
    os.makedirs(directory + '_' + str(run_adjustment) + '_' + str(fish_consumption) + '_' + str(exposure_var))

Error: results/rresultslme_False_True_UIASMMADMA - No such file or directory.
Error: results/rresultslmer_False_True_UIASMMADMA - No such file or directory.
Error: results/rresultsglm_False_True_UIASMMADMA - No such file or directory.
Error: results/rresultsglmer_False_True_UIASMMADMA - No such file or directory.


In [17]:
def fixdf(df):
    
    df['babySex'] = df['babySex'].astype(int)
    df['smoking'] = df['smoking'].astype(int)
    df['education'] = df['education'].astype(int)
    df['parity'] = df['parity'].astype(int)
    
    return df

In [18]:
# dictonaries to hold unadjusted data frames for analysis
frames_to_r_indv = dict()
bin_frames_to_r_indv = dict()
frames_to_r_all = dict()
bin_frames_to_r_all = dict()

#d_test = df_NEU[['PIN_Patient','CohortType','race', 'education','babySex','BMI', 'ga_collection','birth_year','age','SPECIFICGRAVITY_V2']]
#all_vars = covars + [x_feature] 
Y_features_continuous = ['Outcome_weeks','birthWt', 'headCirc', 'birthLen']
Y_features_binary    =  ['LGA','Outcome','SGA']


outputs_conf = []
outputs_crude = []

## loop over all outcomes
for outcome in Y_features_binary + Y_features_continuous:
    
    for name, df_coh in frames_for_analysis:
        print('Working on ', name)

        #variables for fitting procedure
        cat_vars = ['babySex','smoking','education']
        
        if outcome in Y_features_binary:
            contin_vars = ['PIN_Patient','BMI',exposure_var,'parity'] + [outcome]
        if outcome in Y_features_continuous:
            contin_vars = ['PIN_Patient','BMI',exposure_var,'parity'] + [outcome]
        
        # dummy code
        #df_coh_coded_model =  dummy_code(df_coh, cat_vars, contin_vars)
        
        vars_keep = [outcome, 'CohortType'] + cat_vars + [x for x in contin_vars if outcome != x]
        df_model = df_coh[vars_keep]
        
        
        ## variables for addjustment procedure
        ##adjust_cat_vars =  ['babySex','smoking','education','race']
        ##adjust_contin_vars = ['PIN_Patient','CohortType','BMI', 'ga_collection','birth_year','age']
        
        ## loop over cohorts and merge with one-hot encoded data and adjustments
        
        if name in ['NEU', 'UNM', 'DAR']:

            if outcome in Y_features_continuous and name != 'ALL':

                fin = df_model

                fin = fin.dropna()
                fin = fixdf(fin)
                
                fin[exposure_var] = np.log(fin[exposure_var])
                frames_to_r_indv[name + '|' + outcome ] = fin
              
                    
                
            if outcome in Y_features_binary and name != 'ALL':
                
              
                fin = df_model
                fin = fin.dropna()
                fin = fixdf(fin)
                fin[exposure_var] = np.log(fin[exposure_var])
                bin_frames_to_r_indv[name + '|' + outcome] = fin

            
            
        if name in ['ALL']:
            
          
            if outcome in Y_features_continuous and name == 'ALL':

              
                fin = df_model
                fin = fin.dropna()
                fin = fixdf(fin)
                fin[exposure_var] = np.log(fin[exposure_var])
                frames_to_r_all[name + '|' + outcome ] = fin
                
                    
            if outcome in Y_features_binary and name == 'ALL':

                
                fin = df_model
                fin = fin.dropna()
                fin = fixdf(fin)
                fin[exposure_var] = np.log(fin[exposure_var])
                bin_frames_to_r_all[name + '|' + outcome] = fin
                

Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL
Working on  UNM
Working on  DAR
Working on  ALL


/tmp/ipykernel_22355/171439228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['babySex'] = df['babySex'].astype(int)
/tmp/ipykernel_22355/171439228.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smoking'] = df['smoking'].astype(int)
/tmp/ipykernel_22355/171439228.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [19]:
frames_to_r_all['ALL|Outcome_weeks'].head()

,Outcome_weeks,CohortType,babySex,smoking,education,PIN_Patient,BMI,UIASMMADMA,parity
0,41.098400,UNM,2,0,2,1,55.809009,0.158488,5
1,41.441250,UNM,2,0,2,1,66.086658,NaN,5
4,34.187279,UNM,2,0,2,3,66.749766,0.280016,4
5,34.373881,UNM,2,0,2,3,74.931595,-0.898364,4
6,36.554387,UNM,1,0,2,4,81.433731,-2.005702,0


# Start R stuff

### Using R because it has better GLMR packages


In [20]:
%load_ext rpy2.ipython


In [21]:
# Analysis for individual data

In [22]:
# Set up output directories for results

outputdir_lme = directories_for_output[0] + '_' + str(run_adjustment) + '_' + str(fish_consumption) + '_' + str(exposure_var)
outputdir_lmer = directories_for_output[1] + '_' + str(run_adjustment) + '_' + str(fish_consumption) + '_' +  str(exposure_var)
outputdir_glm = directories_for_output[2]  + '_' +  str(run_adjustment) + '_' + str(fish_consumption) + '_' +  str(exposure_var)
outputdir_glmer = directories_for_output[3]  + '_' +  str(run_adjustment) + '_' + str(fish_consumption) + '_' +  str(exposure_var)


%Rpush outputdir_lme
%Rpush outputdir_lmer
%Rpush outputdir_glm
%Rpush outputdir_glmer

In [23]:
if exposure_var == 'UTAS':

    data_outcome_weeks_NEU = frames_to_r_indv['NEU|Outcome_weeks']
    data_birthWt_NEU  = frames_to_r_indv['NEU|birthWt']
    data_headCirc_NEU = frames_to_r_indv['NEU|headCirc']
    data_birthLen_NEU = frames_to_r_indv['NEU|birthLen']

    %Rpush data_outcome_weeks_NEU
    %Rpush data_birthWt_NEU
    %Rpush data_headCirc_NEU
    %Rpush data_birthLen_NEU

# either way we need the UNM and DAR data

data_outcome_weeks_DAR = frames_to_r_indv['DAR|Outcome_weeks']
data_birthWt_DAR  = frames_to_r_indv['DAR|birthWt']
data_headCirc_DAR = frames_to_r_indv['DAR|headCirc']
data_birthLen_DAR = frames_to_r_indv['DAR|birthLen']

%Rpush data_outcome_weeks_DAR
%Rpush data_birthWt_DAR
%Rpush data_headCirc_DAR
%Rpush data_birthLen_DAR


data_outcome_weeks_UNM = frames_to_r_indv['UNM|Outcome_weeks']
data_birthWt_UNM  = frames_to_r_indv['UNM|birthWt']
data_headCirc_UNM = frames_to_r_indv['UNM|headCirc']
data_birthLen_UNM = frames_to_r_indv['UNM|birthLen']

%Rpush data_outcome_weeks_UNM
%Rpush data_birthWt_UNM
%Rpush data_headCirc_UNM
%Rpush data_birthLen_UNM

In [24]:
data_outcome_weeks_DAR.shape

(1649, 9)

In [25]:
use_dilution_as_covar = False

In [26]:
if use_dilution_as_covar == False:


    fit_str_outcome_weeks_indv = 'Outcome_weeks ~ parity + factor(babySex) + factor(education) + BMI + {}'.format(exposure_var)

    fit_str_birthWt_indv =  'birthWt ~ parity +  factor(babySex) + factor(education) + BMI + {}'.format(exposure_var) 

    fit_str_headCirc_indv = 'headCirc ~ parity +  factor(babySex) + factor(education) + BMI + {}'.format(exposure_var) 

    fit_str_birthLen_indv = 'birthLen ~ parity +  factor(babySex) + factor(education) + BMI + {}'.format(exposure_var) 
    

if use_dilution_as_covar == True:


    fit_str_outcome_weeks_indv = 'Outcome_weeks ~ parity + babySex + education + BMI + {} + dilutionvar'.format(exposure_var)

    fit_str_birthWt_indv = 'birthWt ~ parity +  babySex + education + BMI + {} + dilutionvar'.format(exposure_var) 

    fit_str_headCirc_indv = 'headCirc ~ parity +  babySex + education + BMI + {} + dilutionvar'.format(exposure_var) 

    fit_str_birthLen_indv = 'birthLen ~ parity +  babySex + education + BMI + {} + dilutionvar'.format(exposure_var) 



%Rpush fit_str_outcome_weeks_indv
%Rpush fit_str_birthWt_indv
%Rpush fit_str_headCirc_indv
%Rpush fit_str_birthLen_indv

In [27]:
## decide which analysis is being run


if exposure_var == 'UTAS':

    analysis_info = \
    [[fit_str_outcome_weeks_indv, data_outcome_weeks_NEU, 'NEU_cohorts_outcome_{}.txt'.format(exposure_var)],
    [fit_str_birthWt_indv, data_birthWt_NEU, 'NEU_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_NEU, 'NEU_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_NEU, 'NEU_cohorts_birthLen_{}.txt'.format(exposure_var) ],
    [fit_str_outcome_weeks_indv, data_outcome_weeks_DAR, 'DAR_cohorts_outcome_{}.txt'.format(exposure_var) ],
    [fit_str_birthWt_indv, data_birthWt_DAR, 'DAR_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_DAR, 'DAR_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_DAR, 'DAR_cohorts_birthLen_{}.txt'.format(exposure_var) ]]
   
    #[fit_str_outcome_weeks_indv, data_outcome_weeks_UNM, 'UNM_cohorts_outcome_{}.txt'.format(exposure_var) ],
    #[fit_str_birthWt_indv, data_birthWt_UNM, 'UNM_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    #[fit_str_headCirc_indv, data_headCirc_UNM, 'UNM_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    #[fit_str_birthLen_indv, data_birthLen_UNM, 'UNM_cohorts_birthLen_{}.txt'.format(exposure_var) ]]

if exposure_var == 'UIAS':

    analysis_info = \
    [[fit_str_outcome_weeks_indv, data_outcome_weeks_DAR, 'DAR_cohorts_outcome_{}.txt'.format(exposure_var) ],
    [fit_str_birthWt_indv, data_birthWt_DAR, 'DAR_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_DAR, 'DAR_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_DAR, 'DAR_cohorts_birthLen_{}.txt'.format(exposure_var) ],
    [fit_str_outcome_weeks_indv, data_outcome_weeks_UNM, 'UNM_cohorts_outcome_{}.txt'.format(exposure_var) ],
    [fit_str_birthWt_indv, data_birthWt_UNM, 'UNM_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_UNM, 'UNM_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_UNM, 'UNM_cohorts_birthLen_{}.txt'.format(exposure_var) ]]
    
if exposure_var == 'UIASMMADMA':

    analysis_info = \
    [[fit_str_outcome_weeks_indv, data_outcome_weeks_DAR, 'DAR_cohorts_outcome_{}.txt'.format(exposure_var) ],
    [fit_str_birthWt_indv, data_birthWt_DAR, 'DAR_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_DAR, 'DAR_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_DAR, 'DAR_cohorts_birthLen_{}.txt'.format(exposure_var) ],
    [fit_str_outcome_weeks_indv, data_outcome_weeks_UNM, 'UNM_cohorts_outcome_{}.txt'.format(exposure_var) ],
    [fit_str_birthWt_indv, data_birthWt_UNM, 'UNM_cohorts_birthWt_{}.txt'.format(exposure_var) ],
    [fit_str_headCirc_indv, data_headCirc_UNM, 'UNM_cohorts_headCirc_{}.txt'.format(exposure_var) ],
    [fit_str_birthLen_indv, data_birthLen_UNM, 'UNM_cohorts_birthLen_{}.txt'.format(exposure_var) ]]

In [28]:
#df

In [29]:
%%R



run_lm <- function(fit_str, df, fileout ) {
    m<-lm(fit_str, data=df)
    sink(paste(outputdir_lme, fileout, sep = '/'))
    print(fit_str)
    print(summary(m))
    print(paste(toString(NROW(df)), " observations.", sep = " "))
    sink()
    print('Done')
}


In [30]:
for fit_str, df, fileout in analysis_info:
    
    
    print(fit_str)
    
    df.describe().transpose().to_csv(outputdir_lme + '/' + fileout.replace('.txt','_')+'info.csv')
    
    %Rpush fit_str
    %Rpush df
    %Rpush fileout

    %R run_lm(fit_str, df, fileout)

Outcome_weeks ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
birthWt ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
headCirc ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
birthLen ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
Outcome_weeks ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
birthWt ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
headCirc ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
birthLen ~ parity +  factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"


In [31]:
# Analysis for individual logistic regression results

if exposure_var == 'UTAS':
    data_Outcome_NEU = bin_frames_to_r_indv['NEU|Outcome']
    data_LGA_NEU = bin_frames_to_r_indv['NEU|LGA']
    data_SGA_NEU = bin_frames_to_r_indv['NEU|SGA']
    
    %Rpush data_Outcome_NEU
    %Rpush data_LGA_NEU
    %Rpush data_SGA_NEU

data_Outcome_DAR = bin_frames_to_r_indv['DAR|Outcome']
data_LGA_DAR = bin_frames_to_r_indv['DAR|LGA']
data_SGA_DAR = bin_frames_to_r_indv['DAR|SGA']

%Rpush data_Outcome_DAR
%Rpush data_LGA_DAR
%Rpush data_SGA_DAR

data_Outcome_UNM = bin_frames_to_r_indv['UNM|Outcome']
data_LGA_UNM = bin_frames_to_r_indv['UNM|LGA']
data_SGA_UNM = bin_frames_to_r_indv['UNM|SGA']

%Rpush data_Outcome_UNM
%Rpush data_LGA_UNM
%Rpush data_SGA_UNM



if use_dilution_as_covar == False:

    fit_str_outcome_indv = 'Outcome ~ parity + factor(babySex) + factor(education) + BMI + {}'.format(exposure_var) 

    fit_str_SGA_indv = 'SGA ~ parity + factor(babySex) + factor(education) + BMI + {}'.format(exposure_var)  

    fit_str_LGA_indv = 'LGA ~ parity + factor(babySex) + factor(education) + BMI + {}'.format(exposure_var)  

if use_dilution_as_covar == True:

    fit_str_outcome_indv = 'Outcome ~ parity + factor(babySex) + factor(education) + BMI + {} + dilutionvar'.format(exposure_var) 

    fit_str_SGA_indv = 'SGA ~ parity + factor(babySex) + factor(education) + BMI + {} + dilutionvar'.format(exposure_var)  

    fit_str_LGA_indv = 'LGA ~ parity + factor(babySex) + factor(education) + BMI + {} + dilutionvar'.format(exposure_var)  


%Rpush fit_str_outcome_indv
%Rpush fit_str_SGA_indv
%Rpush fit_str_LGA_indv
%Rpush outputdir_glm

In [32]:
if exposure_var == 'UTAS':

    analysis_info = [[fit_str_outcome_indv, data_Outcome_NEU, 'NEU_outcome_UTAS.txt'],
                    [fit_str_LGA_indv, data_LGA_NEU, 'NEU_LGA_UTAS.txt'],
                    [fit_str_SGA_indv, data_SGA_NEU, 'NEU_SGA_UTAS.txt'],
                    [fit_str_outcome_indv, data_Outcome_DAR, 'DAR_outcome_UTAS.txt'],
                    [fit_str_LGA_indv, data_LGA_DAR, 'DAR_LGA_UTAS.txt'],
                    [fit_str_SGA_indv, data_SGA_DAR, 'DAR_SGA_UTAS.txt']]
                    #[fit_str_outcome_indv, data_Outcome_UNM, 'UNM_outcome_UTAS.txt'],
                   # [fit_str_LGA_indv, data_LGA_UNM, 'UNM_LGA_UTAS.txt'],
                   # [fit_str_SGA_indv, data_SGA_UNM, 'UNM_SGA_UTAS.txt']]
    
if exposure_var == 'UIAS':
    
    analysis_info = [[fit_str_outcome_indv, data_Outcome_DAR, 'DAR_outcome_{}.txt'.format(exposure_var)],
                [fit_str_LGA_indv, data_LGA_DAR, 'DAR_LGA_{}.txt'.format(exposure_var)],
                [fit_str_SGA_indv, data_SGA_DAR, 'DAR_SGA_{}.txt'.format(exposure_var)],
                [fit_str_outcome_indv, data_Outcome_UNM, 'UNM_outcome_{}.txt'.format(exposure_var)],
                [fit_str_LGA_indv, data_LGA_UNM, 'UNM_LGA_{}.txt'.format(exposure_var)],
                [fit_str_SGA_indv, data_SGA_UNM, 'UNM_SGA_{}.txt'.format(exposure_var)]]

if exposure_var == 'UIASMMADMA':
    
    analysis_info = [[fit_str_outcome_indv, data_Outcome_DAR, 'DAR_outcome_{}.txt'.format(exposure_var)],
                [fit_str_LGA_indv, data_LGA_DAR, 'DAR_LGA_{}.txt'.format(exposure_var)],
                [fit_str_SGA_indv, data_SGA_DAR, 'DAR_SGA_{}.txt'.format(exposure_var)],
                [fit_str_outcome_indv, data_Outcome_UNM, 'UNM_outcome_{}.txt'.format(exposure_var)],
                [fit_str_LGA_indv, data_LGA_UNM, 'UNM_LGA_{}.txt'.format(exposure_var)],
                [fit_str_SGA_indv, data_SGA_UNM, 'UNM_SGA_{}.txt'.format(exposure_var)]]
    

In [33]:
#NEU

In [34]:
%%R

library(lmerTest)

run_glm <- function(fit_str, df, fileout ) {
    m<-glm(fit_str, data=df, family = 'binomial')
    sink(paste(outputdir_glm, fileout, sep = '/'))
    print(fit_str)
    print(summary(m))
    print(paste(toString(NROW(df)), " observations.", sep = " "))
    sink()
    print('Done')
}


R[write to console]: Loading required package: lme4

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘lmerTest’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: The following object is masked from ‘package:stats’:

    step




In [35]:
for fit_str, df, fileout in analysis_info:
    
    print(fit_str)
    df.describe().transpose().to_csv(outputdir_glm + '/' + fileout.replace('.txt','_')+'info.csv')
    
    %Rpush fit_str
    %Rpush df
    %Rpush fileout

    %R run_glm(fit_str, df, fileout)

Outcome ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
LGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
SGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
Outcome ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
LGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"
SGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA
[1] "Done"


In [36]:
# Analysis for combined DATA

In [37]:
data_outcome_weeks = frames_to_r_all['ALL|Outcome_weeks']
data_birthWt = frames_to_r_all['ALL|birthWt']
data_headCirc = frames_to_r_all['ALL|headCirc']
data_birthLen = frames_to_r_all['ALL|birthLen']

%Rpush data_outcome_weeks
%Rpush data_birthWt
%Rpush data_headCirc
%Rpush data_birthLen

In [38]:
%Rpush outputdir_lmer

In [39]:
if use_dilution_as_covar == False:

    fit_str_outcome_weeks = 'Outcome_weeks ~ parity + factor(babySex) + factor(education)  + BMI + {} + (1|CohortType)'.format(exposure_var)  

    fit_str_birthWt = 'birthWt ~ parity + factor(babySex) + factor(education) + BMI + {} + (1|CohortType)'.format(exposure_var)  

    fit_str_headCirc = 'headCirc ~ parity + factor(babySex) + factor(education) + BMI + {} + (1|CohortType)'.format(exposure_var) 

    fit_str_birthLen = 'birthLen ~ parity + factor(babySex) + factor(education) + BMI + {} + (1|CohortType)'.format(exposure_var) 

if use_dilution_as_covar:

    fit_str_outcome_weeks = 'Outcome_weeks ~ parity + babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var)  

    fit_str_birthWt = 'birthWt ~ parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var)  

    fit_str_headCirc = 'headCirc ~ parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var) 

    fit_str_birthLen = 'birthLen ~ parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var) 



%Rpush fit_str_outcome_weeks
%Rpush fit_str_birthWt
%Rpush fit_str_headCirc
%Rpush fit_str_birthLen

In [40]:
analysis_info = [[fit_str_outcome_weeks, data_outcome_weeks, 'all_cohorts_outcome_weeks_{}.txt'.format(exposure_var)],
                [fit_str_birthWt, data_birthWt, 'all_cohorts_birthWt_{}.txt'.format(exposure_var)],
                [fit_str_headCirc, data_headCirc, 'all_cohorts_headCirc_{}.txt'.format(exposure_var)],
                [fit_str_birthLen, data_birthLen, 'all_cohorts_birthLen_{}.txt'.format(exposure_var)]]

In [41]:
%%R

library(lmerTest)

run_lmer <- function(fit_str, df, fileout ) {
    m<-lmer(fit_str, data=df)
    sink(paste(outputdir_lmer, fileout, sep = '/'))
    print(fit_str)
    print(summary(m))
    print(paste(toString(NROW(df)), " observations.", sep = " "))
    sink()
    print('Done')
}


In [42]:
for fit_str, df, fileout in analysis_info:
    
    print(fit_str)
    df.describe().transpose().to_csv(outputdir_lmer + '/' +   fileout.replace('.txt','_')+'info.csv')
    
    %Rpush fit_str
    %Rpush df
    %Rpush fileout

    %R run_lmer(fit_str, df, fileout)

Outcome_weeks ~ parity + factor(babySex) + factor(education)  + BMI + UIASMMADMA + (1|CohortType)
[1] "Done"
birthWt ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA + (1|CohortType)
[1] "Done"
headCirc ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA + (1|CohortType)
[1] "Done"


R[write to console]: boundary (singular) fit: see help('isSingular')



birthLen ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA + (1|CohortType)
[1] "Done"


In [43]:
# binomial 

## TODO

necessariy imports:

import rpy2
import pandas as pd
%load_ext rpy2.ipython
%R require("ggplot2")
creating dummy data frames:

df1 = pd.DataFrame({"A": [1, 2, 3], "B": [1, 2, 3]})
df2 = pd.DataFrame({"A": [3, 2, 1], "B": [1, 2, 3]})
writing the plotting function:

%%R
plot_gg <- function(df) {
    p <- ggplot(data=df) + geom_line(aes(x=A, y=B))
    print(p)
}
and finally plot with only two lines of code:

for df in df1, df2:
    %Rpush df
    %R plot_gg(df)

In [44]:
data_Outcome = bin_frames_to_r_all['ALL|Outcome']
data_LGA = bin_frames_to_r_all['ALL|LGA']
data_SGA = bin_frames_to_r_all['ALL|SGA']

#data_Outcome.loc[0:50,'CohortType'] = 'NEU2'

%Rpush data_Outcome
%Rpush data_LGA
%Rpush data_SGA

if use_dilution_as_covar == False:

    fit_str_outcome = 'Outcome ~ parity + factor(babySex) + factor(education)+ BMI + {} + (1|CohortType)'.format(exposure_var)

    fit_str_SGA = 'SGA ~ parity + factor(babySex) + factor(education) + BMI + {} + (1|CohortType)'.format(exposure_var) 

    fit_str_LGA = 'LGA ~ parity + factor(babySex) + factor(education) + BMI + {} + (1|CohortType)'.format(exposure_var) 

if use_dilution_as_covar:

    fit_str_outcome = 'Outcome ~parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var)

    fit_str_SGA = 'SGA ~ parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var) 

    fit_str_LGA = 'LGA ~ parity +  babySex_2.0 + education_2.0 + education_3.0 + education_4.0 + \
    education_5.0 + BMI + {} + (1|CohortType) + dilutionvar'.format(exposure_var) 


%Rpush fit_str_outcome
%Rpush fit_str_SGA
%Rpush fit_str_LGA
%Rpush outputdir_glmer


analysis_info = [[fit_str_outcome, data_Outcome, 'all_cohorts_Outcome_{}.txt'.format(exposure_var)],
                [fit_str_SGA, data_SGA, 'all_cohorts_SGA_{}.txt'.format(exposure_var)],
                [fit_str_LGA, data_LGA, 'all_cohorts_LGA_{}.txt'.format(exposure_var)]]

In [45]:
%%R

library(lme4)

run_glmer <- function(fit_str, df, fileout ) {
    m<-glmer(fit_str, data=df, family = binomial)
    sink(paste(outputdir_glmer, fileout, sep = '/'))
    print(fit_str)
    print(summary(m))
    print(paste(toString(NROW(df)), " observations.", sep = " "))
    sink()
}


In [46]:
outputdir_glmer + '/' +  fileout.replace('.txt','_')+'info.csv'

'results/rresultsglmer_False_True_UIASMMADMA/all_cohorts_birthLen_UIASMMADMA_info.csv'

In [47]:
for fit_str, df, fileout in analysis_info:
    
    print(fit_str)
    df.describe().transpose().to_csv(outputdir_glmer + '/' +  fileout.replace('.txt','_')+'info.csv')
    
    %Rpush fit_str
    %Rpush df
    %Rpush fileout

    %R run_glmer(fit_str, df, fileout)

Outcome ~ parity + factor(babySex) + factor(education)+ BMI + UIASMMADMA + (1|CohortType)
SGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA + (1|CohortType)


R[write to console]: boundary (singular) fit: see help('isSingular')



LGA ~ parity + factor(babySex) + factor(education) + BMI + UIASMMADMA + (1|CohortType)


R[write to console]: boundary (singular) fit: see help('isSingular')

